In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [87.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/mar

In [2]:
spark

In [0]:
df=spark.read.csv('multilabel_train.csv',inferSchema=True,header=True)

In [5]:
df.show(5)

+--------------------+--------------------+-----+------------+-------+------+------+-------------+
|                  id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+
|    0000997932d777bf|         Explanation| null|        null|   null|  null|  null|         null|
|Why the edits mad...| just closure on ...|    0|           0|      0|     0|     0|            0|
|    000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|    000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|    0001b41b1c6bb37e|                   "| null|        null|   null|  null|  null|         null|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows



In [7]:
import pyspark.sql.functions as F
df=df.withColumn('idlength',F.length(F.col('id')))
df=df.na.drop()
df.show(5)

+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|                  id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|Why the edits mad...| just closure on ...|    0|           0|      0|     0|     0|            0|      98|
|    000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|    000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|    0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|Congratulations f...| use the tools we...|    0|           0|      0|     0|     0|            0|      31|
+--------------------+--------------------+-----+------------+-------+------+------+-------------+--------+
only showing top 5 rows



In [0]:
df=df.where(F.col('idlength')==16)

In [9]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|      16|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|      16|
|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0|     0|            0|      16|
|00040093b2687caa|alignment on this...|    0|           0|      

In [0]:
df=df.drop('words')

In [11]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|idlength|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|      16|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|      16|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|      16|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|      16|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|      16|
|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0|     0|            0|      16|
|00040093b2687caa|alignment on this...|    0|           0|      

In [0]:
df=df.drop('idlength')
import pyspark.sql.functions as F
df=df.where(F.length('comment_text')>5)

In [0]:
df=df.withColumn("lowered_text",F.lower(F.col("comment_text")))

In [15]:
df.show(20)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|your vandalism to...|
|00037261f536c51d|Sorry if the word...|    0|           0|      0|     0

In [0]:
df=df.cache()

In [17]:
df.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|
|00031b1e95af7921|Your vandalism to...|    0|           0|      0|     0|     0|            0|your vandalism to...|
+----------------+--------------------+-----+------------+-------+------

In [0]:
from pyspark.ml.feature import RegexTokenizer
tokenizer=RegexTokenizer(inputCol='lowered_text',outputCol="tokens",pattern="\W+")

In [0]:
df=tokenizer.transform(df)

In [20]:
df.show(10)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|[cocksucker, befo...|
|00031b1e95af7921|Your vandalism to...|  

In [21]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)
 |-- lowered_text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
from pyspark.ml.feature import CountVectorizer
vectorizer=CountVectorizer(inputCol="tokens",outputCol="countvectorized")

In [0]:
df=df.na.drop()

In [24]:
df.show(30)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|[cocksucker, befo...|
|00031b1e95af7921|Your vandalism to...|  

In [0]:
df=vectorizer.fit(df).transform(df)

In [26]:
df.show(25)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(44404,[1,11,22,4...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(44404,[0,1,2,5,6...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(44404,[3,9,15,16...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|       

In [27]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)
 |-- lowered_text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- countvectorized: vector (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType
for col in df.columns[2:8]:
  df=df.withColumn(col,df[col].cast(IntegerType()))

In [31]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: integer (nullable = true)
 |-- severe_toxic: integer (nullable = true)
 |-- obscene: integer (nullable = true)
 |-- threat: integer (nullable = true)
 |-- insult: integer (nullable = true)
 |-- identity_hate: integer (nullable = true)
 |-- lowered_text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- countvectorized: vector (nullable = true)



In [32]:
df.show()

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(44404,[1,11,22,4...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(44404,[0,1,2,5,6...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(44404,[3,9,15,16...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|       

In [0]:
df=df.filter(~df.id.contains(' '))

In [34]:
df.show(20)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|d'aww! he matches...|[d, aww, he, matc...|(44404,[1,11,22,4...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(44404,[0,1,2,5,6...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(44404,[3,9,15,16...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|       

In [35]:
df.columns[2:8]

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
train,test=df.randomSplit([0.8,0.2],22)

In [0]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [0]:
modelEvaluator=BinaryClassificationEvaluator()
toxic_classifier=LinearSVC(featuresCol='countvectorized',labelCol='toxic',predictionCol='is_toxic',rawPredictionCol='raw_is_toxic')
severe_toxic_classifier=LinearSVC(featuresCol='countvectorized',labelCol='severe_toxic',predictionCol='is_severe_toxic',rawPredictionCol='raw_is_severe_toxic')
obscene_classifier=LinearSVC(featuresCol='countvectorized',labelCol='obscene',predictionCol='is_obscene',rawPredictionCol='raw_is_obscene')
threat_classifier=LinearSVC(featuresCol='countvectorized',labelCol='threat',predictionCol='is_threat',rawPredictionCol='raw_is_threat')
insult_classifier=LinearSVC(featuresCol='countvectorized',labelCol='insult',predictionCol='is_insult',rawPredictionCol='raw_is_insult')
identity_hate_classifier=LinearSVC(featuresCol='countvectorized',labelCol='identity_hate',predictionCol='is_identity_hate',rawPredictionCol='raw_is_identity_hate')

In [0]:
pipeline=Pipeline(stages=[toxic_classifier,severe_toxic_classifier,obscene_classifier,threat_classifier,insult_classifier,identity_hate_classifier])

In [63]:
pipeline

Pipeline_4e52a1815e5cd1f9c09e

In [46]:
train.count()

24593

In [47]:
test.count()

6132

In [0]:
train=train.na.drop()

In [51]:
train.show(50)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|hey man, i'm real...|[hey, man, i, m, ...|(44404,[0,1,2,5,6...|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|you, sir, are my ...|[you, sir, are, m...|(44404,[3,9,15,16...|
|0002bcb3da6cb337|COCKSUCKER BEFORE...|    1|           1|      1|     0|     1|            0|cocksucker before...|[cocksucker, befo...|(44404,[3,15,26,1...|
|00031b1e95af7921|Your vandalism to...|    0|       

In [0]:
train=train.cache()
test=test.cache()

In [64]:
print('Training...')
pipeline_fitted=pipeline.fit(train)
print('Done training.')
print('Generating predictions...')
prediction=pipeline_fitted.transform(test)
print('Predictions made.')

Training...
Done training.
Generating predictions...
Predictions made.


In [66]:
prediction.show(30)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+---------------+--------------------+----------+--------------------+---------+--------------------+---------+--------------------+----------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|        lowered_text|              tokens|     countvectorized|        raw_is_toxic|is_toxic| raw_is_severe_toxic|is_severe_toxic|      raw_is_obscene|is_obscene|       raw_is_threat|is_threat|       raw_is_insult|is_insult|raw_is_identity_hate|is_identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+---------------+--------------------+----------+--------------------+---------